In [1]:
%matplotlib inline
from os.path import join as opj
import json
from nipype.interfaces.spm import Level1Design, EstimateModel, EstimateContrast
from nipype.algorithms.modelgen import SpecifySPMModel
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype import Workflow, Node
import glob
import nipype.interfaces.io as nio
import nipype.interfaces.fsl as fsl
import nipype.interfaces.utility as util
import nipype.pipeline.engine as pe
import nipype.algorithms.modelgen as model

In [2]:
experiment_dir = 'mydataset/output/'
output_dir = 'mydataset/datasink/'
working_dir = 'mydataset/'

# list of subject identifiers
subject_list = [i[13:] for i in glob.glob('fmriprep/sub-*/')]

# TR of functional images
with open('/data/projects/STUDIES/social_doors_college/bids/sub-3836/func/sub-3836_task-srDoorsa_run-01_bold.json', 'rt') as fp:
    task_info = json.load(fp)
TR = task_info['RepetitionTime']

# Smoothing withds used during preprocessing
fwhm = [6]

In [3]:
modelfit = pe.Workflow(name='modelfit')

In [4]:
modelspec = pe.Node(interface=model.SpecifyModel(), name='modelspec')

In [5]:
level1design = pe.Node(interface=fsl.Level1Design(), name='level1design')

In [6]:
modelgen = pe.MapNode(
    interface=fsl.FEATModel(),
    name='modelgen',
    iterfield=['fsf_file','ev_files'])

In [7]:
modelestimate = pe.MapNode(
    interface=fsl.FILMGLS(smooth_autocorr=True, mask_size=5, threshold=1000),
    name='modelestimate',
    iterfield=['design_file', 'in_file'])

In [9]:
conestimate = pe.MapNode(
    interface=fsl.ContrastMgr(),
    name='conestimate',
    iterfield=[
        'tcon_file', 'param_estimates', 'sigmasquareds', 'corrections',
        'dof_file'
    ])

modelfit.connect([
    (modelspec, level1design, [('session_info', 'session_info')]),
    (level1design, modelgen, [('fsf_files', 'fsf_file'), ('ev_files',
                                                          'ev_files')]),
    (modelgen, modelestimate, [('design_file', 'design_file')]),
    (modelgen, conestimate, [('con_file', 'tcon_file')]),
    (modelestimate, conestimate,
     [('param_estimates', 'param_estimates'), ('sigmasquareds',
                                               'sigmasquareds'),
                                      ('dof_file', 'dof_file')]),
])

In [10]:
fixed_fx = pe.Workflow(name='fixedfx')

In [11]:
copemerge = pe.MapNode(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="copemerge")

varcopemerge = pe.MapNode(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="varcopemerge")

In [12]:
level2model = pe.Node(interface=fsl.L2Model(), name='l2model')

In [13]:
flameo = pe.MapNode(
    interface=fsl.FLAMEO(run_mode='fe'),
    name="flameo",
    iterfield=['cope_file', 'var_cope_file'])

fixed_fx.connect([
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level2model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), ('design_grp',
                                                          'cov_split_file')]),
])

In [20]:
def sort_copes(files):
    numelements = len(files[0])
    outfiles = []
    for i in range(numelements):
        outfiles.insert(i, [])
        for j, elements in enumerate(files):
            outfiles[i].append(elements[i])
    return outfiles


def num_copes(files):
    return len(files)


firstlevel = pe.Workflow(name='firstlevel')
